## Load data

In [1]:
import os
import sys
from tqdm import tqdm
import argparse
from pathlib import Path
from dataclasses import dataclass
from typing import Any, Dict, List,Tuple, Optional
import json
#
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)
print('module_path : ', module_path)
from config import SciFactT5Config
import definitions
sys.path.append(os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))
print('definitions.PROJECT_VARS.ROOT_DIR : ', definitions.PROJECT_VARS.ROOT_DIR)
print('os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR) : ',os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))
#

from T5ParEvo.src.data.data import Claim, Label, ClaimPredictions,GoldDataset
from T5ParEvo.src.linguistic.ner_abbr import NEREntity, Abbreviation
from multivers import util
from multivers.data_r import ClaimDataLoaderGenerator, get_dataloader, DataLoaderGenerator
from multivers.model_r import MultiVerSModel

from T5ParEvo.target_system.multivers.multivers_interface import PredictionParams, ModelPredictor

module_path :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
root dir :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
definitions.PROJECT_VARS.ROOT_DIR :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR) :  /home/qudratealahyratu/research/nlp/fact_checking/my_work
/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo


/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `predict`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."


In [3]:
# This dataset to be used only for training
cfg= SciFactT5Config()
ds_train = GoldDataset(cfg.target_dataset.loc_target_dataset_corpus,
                    cfg.target_dataset.loc_target_dataset_train)
claim_train = ds_train.get_claim(39)

In [4]:

params = PredictionParams(
    checkpoint_path= "/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/checkpoints/scifact.ckpt",
    output_file= None,#"prediction/pred_opt_scifact.jsonl",
    batch_size=5,
    device=0,
    num_workers=4,
    no_nei=False,
    force_rationale=False,
    debug=False,
)
corpus_file = cfg.target_dataset.loc_target_dataset_corpus#cfg.target_dataset.loc_target_dataset_test#"/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/corpus.jsonl"

gold_claims = Claim.load_claims_from_file(cfg.target_dataset.loc_target_dataset_test) 


In [5]:
#get unique claims
unique_gold_claims = Claim.get_unique_claims(gold_claims)
# Predict for unique claims
dataloader_generator = DataLoaderGenerator(params, unique_gold_claims, corpus_file)
dataloader = dataloader_generator.get_dataloader_by_claims()
predictor = ModelPredictor(params, dataloader)
prediction_formatted = predictor.run()

Some weights of the model checkpoint at allenai/longformer-large-4096 were not used when initializing LongformerModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Metric` was deprecated since v1.3.0 in favor of `torchmetrics.metric.Metric`. It will be removed in v1.5.0.
  stream(template_mgs % m

In [6]:
# prediction_formatted
claim_org_predictions: List[ClaimPredictions] = []
# format all the predictions
for cur_prediction in tqdm(prediction_formatted, desc="Formatting predictions"):
    cur_claim = Claim.get_claim_by_id(gold_claims, cur_prediction['id'])
    claim_predictions = ClaimPredictions.from_formatted_prediction(cur_prediction, gold_claim = cur_claim)
    claim_org_predictions.append(claim_predictions)

Formatting predictions: 100%|██████████| 297/297 [00:00<00:00, 31281.91it/s]


In [8]:
claim_org_predictions[0].gold.claim

'10-20% of people with severe mental disorder receive no treatment in low and middle income countries.'

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

In [10]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
paraphrasing_tokenizer = T5Tokenizer.from_pretrained("t5-base")
paraphrasing_model = T5ForConditionalGeneration.from_pretrained("coderpotter/T5-for-Adversarial-Paraphrasing")

# this function will take a sentence, top_k and top_p values based on beam search
def generate_paraphrases(sentence, top_k=50, top_p=0.99):
    text = "paraphrase: " + sentence + " </s>"
    encoding = paraphrasing_tokenizer.encode_plus(text, max_length=256, padding="max_length", return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    beam_outputs = paraphrasing_model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=top_k,
        top_p=top_p,
        early_stopping=True,
        num_return_sequences=10,
    )
    final_outputs = []
    for beam_output in beam_outputs:
        sent = paraphrasing_tokenizer.decode(beam_output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)
    return final_outputs

In [15]:
paraphrase_claims = []
for cur_claim in tqdm(claim_org_predictions):
    paraphrases = generate_paraphrases(cur_claim.gold.claim)
    cur_claim_paraphrased = {}
    cur_claim_paraphrased['claim'] = cur_claim.gold.claim
    cur_claim_paraphrased['paraphrases'] = paraphrases
    paraphrase_claims.append(cur_claim_paraphrased)

  0%|          | 0/297 [00:00<?, ?it/s]/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 297/297 [03:26<00:00,  1.43it/s]


In [48]:
paraphrase_claims[26]

{'claim': 'Autologous transplantation of mesenchymal stem cells has worse graft function than induction therapy with anti-interleukin-2 receptor antibodies.',
 'paraphrases': ['Autologous stem cell transplant has worse graft function than induction therapy with anti-interleukin-2 receptor antibodies.',
  'Autologous transplantation of mesenchymal stem cells has worse graft function than induction therapy with anti-Il-A receptor antibodies.',
  'Autologous transplantation of mesenchymal stem cells has worse graft function than the induction therapy with anti-interleukin-2 receptor antibodies.',
  'Autologous stem cells transplanting have worse graft function than Induction Therapy with anti-interleukin-2 receptor antibodies.',
  'Autologous transplantation of mesenchymal stem cells has a worse graft function than induction therapy with anti-Interleukin-2 receptor antibodies.',
  'Autologous mesenchymal stem cell transplant has worse graft function than induction therapy with anti-interl

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("stanford-oval/paraphraser-bart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("stanford-oval/paraphraser-bart-large")

Downloading:   0%|          | 0.00/50.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [ ]:
        outputs = model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['max_length'],
            do_sample=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['do_sample'],
            top_k=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['top_k'],
            top_p=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['top_p'],
            repetition_penalty=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['repetition_penalty'],
            early_stopping=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['early_stopping'],
            num_return_sequences=PARAPHRASE_PROJECT_SETTINGS['paraphrase_model']['t5_paraphrase_model_params']['num_return_sequences']
        )

In [51]:
# Load model directly
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("THUDM/WebGLM", trust_remote_code=True)

Downloading:   0%|          | 0.00/999 [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Downloading:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Downloading:   0%|          | 0.00/38.9k [00:00<?, ?B/s]

ImportError: cannot import name 'add_code_sample_docstrings' from 'transformers.utils' (/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/transformers/utils/__init__.py)